In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import os
import pandas as pd
from termcolor import colored
import numpy as np
from collections import OrderedDict
from collections import Counter
import matplotlib as mpl
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from natsort import natsorted
import sys
import pickle
import seaborn as sns

# Display pandas df without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))



In [3]:
num_robot = 24
fieldsPerKilobot = 6
experiment_time = 1800
num_run = 100

num_areas = 16
fieldsPerArea = 6 #id, posx, posy, color, completed?, #kilo_in_area

In [4]:
completed_task_label = ["time", "areaID", "creation_time", "completion_time", "type", "kID_on_top", "kIDs"]

In [ ]:
results_folders = ['/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/brownian/Regions']
results_folders += ['/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/persistent/Regions']
results_folders += ['/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/adaptive/Regions']

# results_folders = ['/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/brownian/Mixed']
# results_folders += ['/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/persistent/Mixed']
# results_folders += ['/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/adaptive/Mixed']

## NON ADAPTIVE timeout

In [ ]:
pareto_dict = {}

for results_folder in results_folders:
    
    pareto_parameters = np.array([]).astype(int)
    
    for timeout_folder in natsorted(os.listdir(os.path.join(results_folder))):
        timeout = -1
        print(colored("Timeout folder:", 'blue'),timeout_folder)

        parameters = timeout_folder.split("_")

#         print("parameters:",parameters)
        for param in parameters:
            if param.startswith("timeout"):
    #             print(param.split("#")[-1])
                timeout = int(param.split("#")[-1]) * 10
#                 print("\ttimeout:",timeout)


        if timeout == -1 :
            print(colored("\tWARNING: wrong timeout folder", 'green'))
            continue

        exp_config = parameters[0]+"_"+parameters[1]+"_"+str(timeout)
        print("\tExp configuration:",exp_config)
        pareto_dict[exp_config] = {}
        
        
        for filename in os.listdir(os.path.join(results_folder, timeout_folder)):
            if filename.endswith("completed_taskLOG.tsv"):
                if not os.path.getsize(os.path.join(results_folder, timeout_folder, filename)) > 0:
                    print(colored("\tWARNING, empty file at:" + filename, 'red'))
                    continue
#                 print('\tfilename: ', filename)

                df_completedTask = pd.read_csv(os.path.join(results_folder, timeout_folder, filename), sep="\t", header=None)
                df_completedTask.columns = completed_task_label


                df_blue = df_completedTask.where(df_completedTask['type'] == 0)
                filtered_blue = df_blue[df_blue['time'].notnull()]

                df_red = df_completedTask.mask(df_completedTask['type'] == 0)
                filtered_red = df_red[df_red['time'].notnull()]
                
                num_areas = df_completedTask.shape[0]
                delta_areas = np.abs(filtered_blue.shape[0]-filtered_red.shape[0])
                
#                 print("\tdelta aree:", delta_areas, end="\t")
#                 print("aree completate:",num_areas)
                
                pareto_parameters = np.vstack([pareto_parameters, np.array([delta_areas, num_areas])]) if pareto_parameters.size else np.array([delta_areas, num_areas])
        
#         print("np.mean(pareto_parameters, axis=0):",np.mean(pareto_parameters, axis=0))     
        pareto_dict[exp_config] = np.mean(pareto_parameters, axis=0)

#         break


In [ ]:
# pareto_parameters

## ADAPTIVE timeout

In [10]:
# results_folder = "/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/adaptiveTimeout/Mixed"
results_folder = "/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/adaptiveTimeout/Regions"

pareto_adaptiveTimeout_dict = {}

pareto_adaptiveTimeout_parameters = np.array([]).astype(int)

for timeout_folder in natsorted(os.listdir(os.path.join(results_folder))):
    timeout = -1
    print(colored("Timeout folder:", 'blue'),timeout_folder)

    parameters = timeout_folder.split("_")

#         print("parameters:",parameters)
    for param in parameters:
        if param.startswith("timeout"):
#             print(param.split("#")[-1])
            timeout = int(param.split("#")[-1]) * 10
#                 print("\ttimeout:",timeout)


    if timeout == -1 :
        print(colored("\tWARNING: wrong timeout folder", 'green'))
        continue

    exp_config = parameters[0]+"_"+parameters[1]+"_"+str(timeout)
    print("\tExp configuration:",exp_config)
    pareto_adaptiveTimeout_dict[exp_config] = {}

    for filename in os.listdir(os.path.join(results_folder, timeout_folder)):
        if filename.endswith("completed_taskLOG.tsv"):
            if not os.path.getsize(os.path.join(results_folder, timeout_folder, filename)) > 0:
                print(colored("\tWARNING, empty file at:" + filename, 'red'))
                continue
#             print('\tfilename: ', filename)

            df_completedTask = pd.read_csv(os.path.join(results_folder, timeout_folder, filename), sep="\t", header=None)
            df_completedTask.columns = completed_task_label


            df_blue = df_completedTask.where(df_completedTask['type'] == 0)
            filtered_blue = df_blue[df_blue['time'].notnull()]

            df_red = df_completedTask.mask(df_completedTask['type'] == 0)
            filtered_red = df_red[df_red['time'].notnull()]

            num_areas = df_completedTask.shape[0]
            delta_areas = np.abs(filtered_blue.shape[0]-filtered_red.shape[0])

            print("\t|red-blue|:", delta_areas, end="\t")
            print("aree completate:",num_areas)

            pareto_adaptiveTimeout_parameters = np.vstack([pareto_adaptiveTimeout_parameters, np.array([delta_areas, num_areas])]) if pareto_adaptiveTimeout_parameters.size else np.array([delta_areas, num_areas])

    print("np.mean(pareto_adaptiveTimeout_parameters, axis=0):",np.mean(pareto_adaptiveTimeout_parameters, axis=0))     
    pareto_adaptiveTimeout_dict[exp_config] = np.mean(pareto_adaptiveTimeout_parameters, axis=0)


#         break


Timeout folder: Regions_Adaptive_2021-07-16_robots#24_timeout#6_respawn#30_NumAreas#16_redAreas#8_1800#seconds
	Exp configuration: Regions_Adaptive_60
	|red-blue|: 3	aree completate: 69
	|red-blue|: 16	aree completate: 68
	|red-blue|: 2	aree completate: 68
	|red-blue|: 12	aree completate: 78
	|red-blue|: 18	aree completate: 68
	|red-blue|: 26	aree completate: 86
	|red-blue|: 18	aree completate: 68
	|red-blue|: 10	aree completate: 68
	|red-blue|: 27	aree completate: 69
	|red-blue|: 1	aree completate: 73
	|red-blue|: 12	aree completate: 72
	|red-blue|: 15	aree completate: 75
	|red-blue|: 19	aree completate: 61
	|red-blue|: 20	aree completate: 76
	|red-blue|: 9	aree completate: 73
	|red-blue|: 18	aree completate: 74
	|red-blue|: 26	aree completate: 60
	|red-blue|: 25	aree completate: 65
	|red-blue|: 11	aree completate: 73
	|red-blue|: 13	aree completate: 65
	|red-blue|: 21	aree completate: 85
	|red-blue|: 12	aree completate: 72
	|red-blue|: 1	aree completate: 57
	|red-blue|: 30	aree compl

	|red-blue|: 114	aree completate: 148
	|red-blue|: 74	aree completate: 112
	|red-blue|: 81	aree completate: 115
	|red-blue|: 74	aree completate: 112
	|red-blue|: 69	aree completate: 111
	|red-blue|: 55	aree completate: 97
	|red-blue|: 64	aree completate: 120
	|red-blue|: 88	aree completate: 128
	|red-blue|: 103	aree completate: 137
	|red-blue|: 85	aree completate: 121
	|red-blue|: 90	aree completate: 134
	|red-blue|: 80	aree completate: 118
	|red-blue|: 59	aree completate: 109
	|red-blue|: 76	aree completate: 110
	|red-blue|: 67	aree completate: 109
	|red-blue|: 57	aree completate: 109
	|red-blue|: 81	aree completate: 125
	|red-blue|: 87	aree completate: 111
	|red-blue|: 62	aree completate: 114
	|red-blue|: 83	aree completate: 115
	|red-blue|: 82	aree completate: 124
	|red-blue|: 91	aree completate: 115
	|red-blue|: 78	aree completate: 106
	|red-blue|: 92	aree completate: 124
	|red-blue|: 89	aree completate: 123
	|red-blue|: 75	aree completate: 109
	|red-blue|: 74	aree completate: 120


In [ ]:
pareto_adaptiveTimeout_dict

# Pareto diagram

In [ ]:
markers=['o', '^', 's']
colors=['b', 'r', 'g']
alphas = np.linspace(0.2, 1, num=12)

plt.figure(figsize=(15, 10), dpi=80)


plt.rcParams.update({'font.size': 18})

brownian = mlines.Line2D([], [], color='b', marker='o', linestyle='None',
                          markersize=8, label='Brownian walk')
adaptive = mlines.Line2D([], [], color='g', marker='s', linestyle='None',
                          markersize=8, label='Adaptive walk')
persistent = mlines.Line2D([], [], color='r', marker='^', linestyle='None',
                          markersize=8, label='Persistent walk')

adaptiveTimeoutBrownian = mlines.Line2D([], [], color='b', marker='o', linestyle='None',
                          markersize=8, label='Brownian, adaptive timeout', fillstyle='none')
adaptiveTimeoutAdaptive = mlines.Line2D([], [], color='g', marker='s', linestyle='None',
                          markersize=8, label='Adaptive, adaptive timeout', fillstyle='none')
adaptiveTimeoutPersistent = mlines.Line2D([], [], color='r', marker='^', linestyle='None',
                          markersize=8, label='Persistent, adaptive timeout', fillstyle='none')

plt.legend(handles=[brownian, persistent, adaptive, adaptiveTimeoutBrownian, adaptiveTimeoutPersistent, adaptiveTimeoutAdaptive], loc="lower right")#, fontsize=11)



for alpha,(key,value) in enumerate(pareto_dict.items()):
#     print(key)
    [setup, walk, timeout] = key.split('_')
    
    if walk == "Brownian":
        i = 0
    elif walk == "Persistent":
        i = 1
    else:
        i = 2
    plt.scatter(value[0], value[1],c=colors[i], marker=markers[i], linewidths=3.0, alpha=alphas[alpha%len(alphas)])

    
for key,value in pareto_adaptiveTimeout_dict.items():
    [setup, walk, timeout] = key.split('_')
    if walk == "Brownian":
        i = 0
    elif walk == "Persistent":
        i = 1
    else:
        i = 2
    plt.scatter(value[0], value[1], s=100, facecolors='none', edgecolors=colors[i], marker=markers[i], linewidths=1.5, alpha=1.0)
  

plt.ylabel("Completed areas")
plt.xlabel("Delta red-blue areas")
plt.ylim(-1, 120)
plt.xlim(-1, 120)




# cmap = mpl.cm.gist_gray
# norm = mpl.colors.Normalize(vmin=0.2, vmax=1)

# cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
#                                 norm=norm,
#                                 orientation='horizontal')
# cb1.set_label('Some Units')

N=13
cmap = mpl.cm.gist_gray
norm = mpl.colors.Normalize(vmin=0.0,vmax=1)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cbar = plt.colorbar(sm, ticks=np.linspace(1,0.0,N), label="timeout")#, boundaries=np.arange(-0.05,2.1,.1))
cbar.ax.set_yticklabels(["0","10","30","60","120","180","240","300","360","420", "480", "540", "600"])

plt.savefig(setup+"_pareto_diagram.png")
plt.show()